<a href="https://colab.research.google.com/github/kevgam/CAS_IE_Information_Retrieval/blob/main/IR_Test_NEU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Vorbereitung des Datensatzes in Spark

Unser ursprünglicher Spotify-Datensatz, der über [Kaggle](https://www.kaggle.com/discussions/accomplishments/522912) verfügbar ist, umfasste fast eine Million Datensätze. Aufgrund der Grösse des Datensatzes hatten wir bei der Verarbeitung Performanceprobleme. Daher haben wir den Datensatz zunächst in der Spark-Umgebung der ZHAW vorverarbeitet.


Nach dem Standardlogin (inkl. sc.stop() am Schluss) gemäss Anleitung der ZHAW haben wir folgende Schritte ausgeführt:

### Installation und Test der Umgebung

```python
# Installation des notwendigen Pakets
sparky.installpackage('langdetect')

# Alternativ mit pip
pip install langdetect

# Test der RDD-Funktionalität
import os
liste = range(16)
rdd = sc.parallelize(liste)
print(rdd.collect())
print(rdd.glom().collect())

# Überprüfen, ob alle Worker die notwendige Software installiert haben
if len(list(filter(lambda x: x == [], rdd.glom().collect()))):
    raise SystemExit("Nicht gut - einige Worker bleiben ohne Softwareinstallation.")

# Testfunktion für Abhängigkeiten
def testdep(ignore_arg):
    ip = "160.85.252.66"  # Beispiel-IP
    try:
        import lxml
    except:
        return f"lxml FAILED! @ {ip}"
    else:
        return f"lxml worked @ {ip}"

# Installation von Abhängigkeiten
import subprocess
def installdeps(ignore_arg):
    p = subprocess.run("pip install lxml", shell=True, stdout=subprocess.PIPE)
    return p.stdout.decode()

# Ausführen der Installation und Tests
rdd.map(installdeps).collect()
rdd.map(testdep).collect()


### Laden und Verarbeiten der Daten

```python
# Das File wurde vorgängig in unseren Ordner auf dem Server kopiert
filepath = 'songs_with_attributes_and_lyrics.csv'

# Laden der CSV-Datei
import pandas as pd
dfs = pd.read_csv(filepath)

# Spark DataFrame laden
from pyspark.sql.functions import udf, col
from pyspark.sql.types import StringType
dfs = spark.read.csv(filepath, header=True, inferSchema=True)


### Sprache erkennen und Fortschritt protokollieren

```python
from langdetect import detect

# Funktion zur Erkennung der Sprache mit Fortschrittsanzeige
def detect_language_with_progress(partition):
    total_rows = 0
    for row in partition:
        try:
            lang = detect(row['lyrics'])
            yield (row['lyrics'], lang)  # Rückgabe: Originaltext und erkannte Sprache
        except Exception:
            yield (row['lyrics'], 'unknown')
        total_rows += 1
        if total_rows % 1000 == 0:  # Fortschritt alle 1000 Zeilen anzeigen
            print(f"Processed {total_rows} rows in this partition")

# RDD-Transformationen anwenden
rdd = dfs.rdd.mapPartitions(detect_language_with_progress)

# Zurück in ein DataFrame umwandeln
schema = StringType()
result = rdd.toDF(["lyrics", "lyrics_language"])

# Fortschritt anzeigen
result.show()


### Ergebnisse speichern

```python
# Als Excel-Datei speichern
output_path = './processed_songs_with_lyrics.xlsx'
dfs.to_excel(output_path, index=False)
print(f"DataFrame saved to: {output_path}")

# Als CSV-Datei speichern
csv_output_path = './processed_songs_with_lyrics.csv'
dfs.to_csv(csv_output_path, index=False)
print(f"DataFrame saved as CSV file to: {csv_output_path}")


### Filterung und Speicherung der englischen Texte

```python
# Zeilen filtern, in denen die Sprache Englisch ist
dfs_en = dfs[dfs['lyrics_language'] == 'en']

# Gefilterte Daten als Excel- und CSV-Datei speichern
dfs_en.to_excel('./processed_songs_filtered_lyrics_en.xlsx', index=False)
dfs_en.to_csv('./processed_songs_filtered_lyrics_en.csv', index=False)

print("Filtered DataFrame saved as 'filtered_lyrics_en.xlsx' and 'filtered_lyrics_en.csv'")


# Herausfiltern der Datensätze ohne Albumnamen

Da die